In [ ]:
# Data Sources
## Data files saved in 'Data' folder

GDP by MSA – 2001 -2017 data ( MAGDP files)
Population, Per capita personal income  2001-2017 data by counties (CAIN30 file) (2001-2017)
2003- 2019 Airportwise passengers/flights data at https://www.transtats.bts.gov/Data_Elements.aspx?Data=1

Airquality data by counties
https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual (1980-2019)


In [1]:
import pandas as pd

# Collecting Air Quality indicators

In [2]:
import glob
import os

path = 'Data/AirQuality' # use correct path for Airquality folder
all_files = glob.glob(os.path.join(path, "*.zip"))

df_AirQ = pd.concat((pd.read_csv(f) for f in all_files))

In [3]:
df_AirQ = df_AirQ[['Year', 'State', 'County', 'Days with AQI','Good Days', 'Max AQI','90th Percentile AQI', 'Median AQI'] ]  

In [4]:
df_AirQ.head()

,Year,State,County,Days with AQI,Good Days,Max AQI,90th Percentile AQI,Median AQI
0,2008,Alabama,Baldwin,265,210,133,68,37
1,2008,Alabama,Clay,110,70,93,61,43
2,2008,Alabama,Colbert,277,201,97,64,43
3,2008,Alabama,DeKalb,110,63,87,64,48
4,2008,Alabama,Elmore,245,180,105,74,42


In [5]:
df_AirQ.Year.unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],
      dtype=int64)

In [6]:
df_AirQ.State.unique(), df_AirQ.County.unique()[1:5]

(array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Country Of Mexico', 'Delaware',
        'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
        'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
        'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
        'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
        'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
        'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
        'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
        'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
        'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
        'Wisconsin', 'Wyoming'], dtype=object),
 array(['Clay', 'Colbert', 'DeKalb', 'Elmore'], dtype=object))

In [7]:
df_Air = df_AirQ.loc[(df_AirQ.State == 'California') | (df_AirQ.State == 'New York') | (df_AirQ.State == 'New Jersey')]
df_Air.shape

(1112, 8)

In [8]:
LA_counties = ['Los Angeles', 'Orange']
SF_counties =['Alameda', 'Contra Costa', 'Marin', 'San Francisco', 'San Mateo']
NY_counties = ['Bergen', 'Essex', 'Hudson', 'Hunterdon', 'Middlesex', 'Monmouth',
              'Morris', 'Ocean', 'Passic', 'Somerset', 'Sussex', 'Union', 'Bronx', 'Dutchess',
              'Kings', 'Nasseu', 'New York', 'Orange', 'Putnam', 'Queens', 'Richmond',
              'Rockland', 'Suffolk', 'Westcher']

In [9]:
df_Air = df_Air[df_Air['County'].isin(LA_counties+ SF_counties+ NY_counties)]
df_Air.County.unique()

array(['Alameda', 'Contra Costa', 'Kings', 'Los Angeles', 'Marin',
       'Orange', 'San Francisco', 'San Mateo', 'Bergen', 'Essex',
       'Hudson', 'Hunterdon', 'Middlesex', 'Monmouth', 'Morris', 'Ocean',
       'Union', 'Bronx', 'Dutchess', 'New York', 'Putnam', 'Queens',
       'Richmond', 'Suffolk', 'Rockland'], dtype=object)

In [10]:
import numpy as np
conditions = [(df_Air['County'].isin(LA_counties)), (df_Air['County'].isin(SF_counties)), (df_Air['County'].isin(NY_counties))]
choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_Air['MSA'] =  np.select(conditions, choices)

In [11]:
df_Air.head()

,Year,State,County,Days with AQI,Good Days,Max AQI,90th Percentile AQI,Median AQI,MSA
56,2008,California,Alameda,366,182,205,83,51,SF_MSA
61,2008,California,Contra Costa,366,266,161,77,42,SF_MSA
70,2008,California,Kings,366,68,220,143,80,NY_MSA
72,2008,California,Los Angeles,366,31,227,172,84,LA_MSA
74,2008,California,Marin,363,347,97,44,31,SF_MSA


In [12]:
df_Air['% Good Days'] = 100* df_Air['Good Days']/df_Air['Days with AQI']

In [13]:
df_AirQuality = df_Air.pivot_table(index = ['MSA', 'Year'], values =['Good Days', 'Median AQI', '90th Percentile AQI', '% Good Days'], aggfunc = np.mean)
df_AirQuality.shape

(33, 4)

In [14]:
df_AirQuality.head()

% Good Days  90th Percentile AQI   Good Days  Median AQI
MSA    Year                                                          
LA_MSA 2008    35.974499           115.666667  131.666667   63.333333
       2009    39.726027           103.333333  145.000000   58.333333
       2010    38.492019            96.333333  140.000000   57.333333
       2011    36.255708           100.000000  132.333333   59.000000
       2012    42.531876            97.666667  155.666667   58.000000

# Population and Per Capita Income

In [15]:
df_income_pop = pd.read_excel('Data/Population_Income/CAINC30__ALL_STATES_1969_2017.xlsx')

In [16]:
#Population (persons) 3/, Per capita personal income 4/


df_income = df_income_pop[df_income_pop.LineCode == 110]
df_population = df_income_pop[df_income_pop.LineCode == 100]
df_income.shape, df_population.shape

((3198, 57), (3198, 57))

In [17]:
df_income.tail(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
99084,"""97000""",Rocky Mountain,7,CAINC30,110.0,...,Per capita personal income 4/,Dollars,3602,3956,...,38567,36273,37054,39569,41677,42701,45307,47053,47505,49265
99115,"""98000""",Far West,8,CAINC30,110.0,...,Per capita personal income 4/,Dollars,4570,4855,...,43214,41298,42636,44966,47451,47819,50787,53911,55550,57748


In [18]:
df_income[df_income.GeoName.str.contains("Orange, CA")]

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
7603,"""06059""","Orange, CA",8,CAINC30,110.0,...,Per capita personal income 4/,Dollars,4821,5074,...,50028,46813,49740,51469,55296,54594,57110,61178,62763,65400


In [19]:
LA_counties1 = ['Los Angeles, CA', 'Orange, CA']
SF_counties1 =['Alameda, CA', 'Contra Costa, CA', 'Marin, CA', 'San Francisco, CA', 'San Mateo, CA']
NY_counties1 = ['Bergen, NJ', 'Essex, NJ', 'Hudson, NJ', 'Hunterdon, NJ', 'Middlesex, NJ', 'Monmouth, NJ',
              'Morris, NJ', 'Ocean, NJ', 'Passic, NJ', 'Somerset, NJ', 'Sussex, NJ', 'Union, NY', 'Bronx, NY', 'Dutchess, NY',
              'Kings, NY', 'Nasseu, NY', 'New York, NY', 'Orange, NY', 'Putnam, NY', 'Queens, NY', 'Richmond, NY',
              'Rockland, NY', 'Suffolk, NY', 'Westcher, NY']


In [20]:
df_income = df_income[['GeoName',2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]
df_population = df_population[['GeoName', 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]
df_population.tail()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
98990,Plains,20248815,20392583,20535719,20638502,20749482,20872827,20984812,21082599,21179519,21294107
99021,Southeast,77057840,77814936,78575213,79260002,79953925,80585268,81304675,82104601,82935602,83715076
99052,Southwest,36269039,36899289,37472786,37972888,38521426,39030404,39619086,40243759,40820143,41339800
99083,Rocky Mountain,10609537,10793888,10949084,11079489,11215355,11366966,11514638,11689105,11878018,12055738
99114,Far West,51608614,52167532,52686778,53176406,53679832,54161802,54695081,55249155,55772858,56250544


In [21]:
df_income = df_income[df_income['GeoName'].isin(LA_counties1+ SF_counties1+ NY_counties1)]
df_population = df_population[df_population['GeoName'].isin(LA_counties1+ SF_counties1+ NY_counties1)]


In [22]:
numeric = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
for year in numeric:
    df_income[year] = df_income[year].astype(int)
    df_population[year] = df_population[year].astype(int)
    
df_income.dtypes

GeoName    object
2008        int32
2009        int32
2010        int32
2011        int32
2012        int32
2013        int32
2014        int32
2015        int32
2016        int32
2017        int32
dtype: object

In [23]:
df_income.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
6704,"Alameda, CA",48943,46330,48118,51247,53323,55378,58984,63809,67356,71282
6890,"Contra Costa, CA",58906,54242,54030,57681,61878,61435,64056,69195,72195,76527
7262,"Los Angeles, CA",43431,41869,43569,46439,49459,49010,52130,55366,56851,58419
7324,"Marin, CA",90711,82249,84002,90963,97274,98203,106548,114455,117552,124552
7603,"Orange, CA",50028,46813,49740,51469,55296,54594,57110,61178,62763,65400


In [24]:
import numpy as np
conditions1 = [(df_income['GeoName'].isin(LA_counties1)), 
              (df_income['GeoName'].isin(SF_counties1)), 
              (df_income['GeoName'].isin(NY_counties1))]

conditions2 = [(df_population['GeoName'].isin(LA_counties1)), 
              (df_population['GeoName'].isin(SF_counties1)), 
              (df_population['GeoName'].isin(NY_counties1))]

choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_income['MSA'] =  np.select(conditions1, choices)
df_population['MSA'] =  np.select(conditions2, choices)

In [25]:
df_income.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
6704,"Alameda, CA",48943,46330,48118,51247,53323,55378,58984,63809,67356,71282,SF_MSA
6890,"Contra Costa, CA",58906,54242,54030,57681,61878,61435,64056,69195,72195,76527,SF_MSA
7262,"Los Angeles, CA",43431,41869,43569,46439,49459,49010,52130,55366,56851,58419,LA_MSA
7324,"Marin, CA",90711,82249,84002,90963,97274,98203,106548,114455,117552,124552,SF_MSA
7603,"Orange, CA",50028,46813,49740,51469,55296,54594,57110,61178,62763,65400,LA_MSA


In [26]:
dfi =df_income.pivot_table(index = ['MSA'], values = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], aggfunc = np.median )
dfp =df_population.pivot_table(index = ['MSA'], values = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], aggfunc = np.sum )

In [27]:
df_population

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
6703,"Alameda, CA",1477208,1498539,1513402,1532215,1556648,1582936,1611572,1637141,1653236,1663190,SF_MSA
6889,"Contra Costa, CA",1023344,1037890,1052799,1066182,1079093,1095310,1110971,1126027,1138645,1147439,SF_MSA
7261,"Los Angeles, CA",9735147,9787400,9824490,9885998,9956152,10018604,10072695,10123248,10150558,10163507,LA_MSA
7323,"Marin, CA",248398,250862,252940,255505,256381,258856,261007,261718,261532,260955,SF_MSA
7602,"Orange, CA",2957593,2987177,3017116,3053465,3085386,3113649,3136750,3160576,3177703,3190400,LA_MSA
7850,"San Francisco, CA",791152,801922,805770,816294,830406,841270,853258,866320,876103,884363,SF_MSA
7943,"San Mateo, CA",703830,713617,719899,729022,740644,750345,759156,766779,769561,771410,SF_MSA
56737,"Bergen, NJ",895328,900319,906879,914233,921035,927202,933348,938255,942387,948406,NY_MSA
56892,"Essex, NJ",778165,781943,784694,787414,788942,792560,797078,800206,803876,808285,NY_MSA
56954,"Hudson, NJ",619533,628572,636345,648619,658909,667066,673773,679907,686391,691643,NY_MSA


In [28]:
dfi = dfi.T
dfp = dfp.T

In [29]:
dfi = dfi.reset_index()
dfp = dfp.reset_index()
dfi['MSA'] = 'LA_MSA'
dfp['MSA'] = 'LA_MSA'
dfi.head()

MSA,index,LA_MSA,NY_MSA,SF_MSA,MSA
0,2008,46729.5,48880.5,75468.0,LA_MSA
1,2009,44341.0,47343.0,71694.0,LA_MSA
2,2010,46654.5,48082.0,73739.0,LA_MSA
3,2011,48954.0,50679.0,79872.0,LA_MSA
4,2012,52377.5,52804.0,87986.0,LA_MSA


In [30]:
dfi1 = dfi[['index', 'LA_MSA']]
dfp1 = dfp[['index', 'LA_MSA']]
dfi2 = dfi[['index', 'SF_MSA']]
dfp2 = dfp[['index', 'SF_MSA']]
dfi3 = dfi[['index', 'NY_MSA']]
dfp3 = dfp[['index', 'NY_MSA']]
dfi1.columns  = ['Year', 'Per Capita Income']
dfi2.columns  = ['Year', 'Per Capita Income']
dfi3.columns  = ['Year', 'Per Capita Income']
dfp1.columns  = ['Year', 'Population']
dfp2.columns  = ['Year', 'Population']
dfp3.columns  = ['Year', 'Population']
dfi1['MSA'] = 'LA_MSA'
dfi2['MSA'] = 'SF_MSA'
dfi3['MSA'] = 'NY_MSA'
dfp1['MSA'] = 'LA_MSA'
dfp2['MSA'] = 'SF_MSA'
dfp3['MSA'] = 'NY_MSA'

C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [31]:
dfi = pd.concat([dfi1, dfi2, dfi3], axis =0)
dfi =dfi[['MSA', 'Year', 'Per Capita Income']]
dfp = pd.concat([dfp1, dfp2, dfp3], axis =0)
dfp =dfp[['MSA', 'Year', 'Population']]
dfi = dfi.set_index(['MSA', 'Year'], drop = True)
dfp = dfp.set_index(['MSA', 'Year'], drop = True)
dfp.head()

Population
MSA    Year            
LA_MSA 2008    12692740
       2009    12774577
       2010    12841606
       2011    12939463
       2012    13041538

# Airtraffic - Passengers boarding

In [32]:
import pandas as pd
import glob
import os

path_la = 'Data/Airtraffic/LA' 
all_files_la = glob.glob(os.path.join(path_la, "*.csv"))


path_sf = 'Data/Airtraffic/SF' 
all_files_sf = glob.glob(os.path.join(path_sf, "*.csv"))

path_ny = 'Data/Airtraffic/NY' 
all_files_ny = glob.glob(os.path.join(path_ny, "*.csv"))

df1la = pd.read_csv(all_files_la[0])
df2la = pd.read_csv(all_files_la[1])
df3la = pd.read_csv(all_files_la[2])

df1sf = pd.read_csv(all_files_sf[0])
df2sf = pd.read_csv(all_files_sf[1])
df3sf = pd.read_csv(all_files_sf[2])

df1ny = pd.read_csv(all_files_ny[0])
df2ny = pd.read_csv(all_files_ny[1])
df3ny = pd.read_csv(all_files_ny[2])



In [33]:
def transform(df):
    df = df.rename(columns=df.iloc[0])
    df.drop(0, inplace=True)
    df.columns = ['Year', 'Month', 'DOMESTIC', 'INTERNATIONAL', 'TOTAL']
    df = df[['Year', 'Month', 'TOTAL']]
    df['Passengers'] = df.TOTAL.str.replace(',', '').astype(float)
    df.drop('TOTAL', axis = 1, inplace = True)
    df = df[(df.Month == 'TOTAL') & (df.Year != '2019') & (df.Year != '2002') 
          & (df.Year != '2003')& (df.Year != '2004')& (df.Year != '2005')& 
          (df.Year != '2006')& (df.Year != '2007') ]
    
    return df

In [34]:
df1la = transform(df1la)
df2la = transform(df2la)
df3la = transform(df3la)
df_atraf_la = df1la
df_atraf_la['Passengers'] = df1la.Passengers+df2la.Passengers+df3la.Passengers
df_atraf_la['MSA'] = 'LA_MSA'
df_atraf_la.head(20)

df1sf = transform(df1sf)
df2sf = transform(df2sf)
df3sf = transform(df3sf)
df_atraf_sf = df1sf
df_atraf_sf['Passengers'] = df1sf.Passengers+df2sf.Passengers+df3sf.Passengers
df_atraf_sf['MSA'] = 'SF_MSA'
df_atraf_sf.head(20)

df1ny = transform(df1ny)
df2ny = transform(df2ny)
df3ny = transform(df3ny)
df_atraf_ny = df1ny
df_atraf_ny['Passengers'] = df1ny.Passengers+df2ny.Passengers+df3ny.Passengers
df_atraf_ny['MSA'] = 'NY_MSA'
df_atraf_ny.head(20)

,Year,Month,Passengers,MSA
82,2008,TOTAL,52866278.0,NY_MSA
95,2009,TOTAL,50476621.0,NY_MSA
108,2010,TOTAL,51424463.0,NY_MSA
121,2011,TOTAL,52355837.0,NY_MSA
134,2012,TOTAL,54229063.0,NY_MSA
147,2013,TOTAL,55995808.0,NY_MSA
160,2014,TOTAL,57497612.0,NY_MSA
173,2015,TOTAL,60916294.0,NY_MSA
186,2016,TOTAL,63798888.0,NY_MSA
199,2017,TOTAL,65634288.0,NY_MSA


In [35]:
df_Airtraffic = pd.concat([df_atraf_la, df_atraf_sf, df_atraf_ny])
df_Airtraffic['Year'] = df_Airtraffic['Year'].astype(int)
df_Airtraffic = df_Airtraffic.set_index(['MSA', 'Year'], drop = True)
df_Airtraffic.drop('Month', axis = 1, inplace = True)
df_Airtraffic

Passengers
MSA    Year            
LA_MSA 2008  36273289.0
       2009  34179598.0
       2010  35475341.0
       2011  37011731.0
       2012  37790767.0
       2013  38908822.0
       2014  40931688.0
       2015  43464589.0
       2016  46920468.0
       2017  48525247.0
       2018  50363107.0
SF_MSA 2008  28479144.0
       2009  27161820.0
       2010  28032599.0
       2011  28661968.0
       2012  30250725.0
       2013  30735736.0
       2014  32411808.0
       2015  34533303.0
       2016  36917676.0
       2017  39404268.0
       2018  41493480.0
NY_MSA 2008  52866278.0
       2009  50476621.0
       2010  51424463.0
       2011  52355837.0
       2012  54229063.0
       2013  55995808.0
       2014  57497612.0
       2015  60916294.0
       2016  63798888.0
       2017  65634288.0
       2018  68431481.0

# GDP by MSA

In [36]:
df_gdp = pd.read_csv('Data/GDP_MSA/MAGDP2_2001_2017_ALL_AREAS.csv')

In [37]:
df_gdp.GeoName.unique()

array(['United States (Metropolitan Portion)',
       'Abilene, TX (Metropolitan Statistical Area)',
       'Akron, OH (Metropolitan Statistical Area)',
       'Albany, GA (Metropolitan Statistical Area)',
       'Albany, OR (Metropolitan Statistical Area)',
       'Albany-Schenectady-Troy, NY (Metropolitan Statistical Area)',
       'Albuquerque, NM (Metropolitan Statistical Area)',
       'Alexandria, LA (Metropolitan Statistical Area)',
       'Allentown-Bethlehem-Easton, PA-NJ (Metropolitan Statistical Area)',
       'Altoona, PA (Metropolitan Statistical Area)',
       'Amarillo, TX (Metropolitan Statistical Area)',
       'Ames, IA (Metropolitan Statistical Area)',
       'Anchorage, AK (Metropolitan Statistical Area)',
       'Ann Arbor, MI (Metropolitan Statistical Area)',
       'Anniston-Oxford-Jacksonville, AL (Metropolitan Statistical Area)',
       'Appleton, WI (Metropolitan Statistical Area)',
       'Asheville, NC (Metropolitan Statistical Area)',
       'Athens-Clarke 

In [38]:
LA_MSA = 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)'
SF_MSA = 'San Francisco-Oakland-Hayward, CA (Metropolitan Statistical Area)'
NY_MSA = 'New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)'

df_gdp = df_gdp[df_gdp.Description == 'All industry total']
df_gdp.columns

Index(['GeoFIPS', 'GeoName', 'Region', 'TableName', 'ComponentName', 'Unit',
       'IndustryId', 'IndustryClassification', 'Description', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017'],
      dtype='object')

In [39]:
select_columns = ['GeoName', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017']
df_gdp = df_gdp[select_columns]
df_gdp.shape

(384, 11)

In [40]:
df_gdp = df_gdp[df_gdp.GeoName.isin([LA_MSA, SF_MSA, NY_MSA])]
df_gdp.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
18357,"Los Angeles-Long Beach-Anaheim, CA (Metropolit...",772747.1,741630.0,763975.9,782565.1,820862.6,852034.2,901979.9,967100.1,996431.8,1043735.1
21750,"New York-Newark-Jersey City, NY-NJ-PA (Metropo...",1257147.7,1276469.8,1338450.7,1363858.9,1439232.6,1477043.4,1542763.2,1618365.7,1662671.3,1717712.2
27057,"San Francisco-Oakland-Hayward, CA (Metropolita...",353338.8,331326.0,330153.6,340757.4,366151.4,385451.5,413026.5,445123.5,475416.6,500710.1


In [41]:
import numpy as np
conditions3 = [(df_gdp['GeoName'] == LA_MSA), 
              (df_gdp['GeoName'] == SF_MSA), 
              (df_gdp['GeoName'] == NY_MSA)]


choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_gdp['MSA'] =  np.select(conditions3, choices)
df_gdp.drop('GeoName', axis = 1, inplace = True)

df_gdp

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
18357,772747.1,741630.0,763975.9,782565.1,820862.6,852034.2,901979.9,967100.1,996431.8,1043735.1,LA_MSA
21750,1257147.7,1276469.8,1338450.7,1363858.9,1439232.6,1477043.4,1542763.2,1618365.7,1662671.3,1717712.2,NY_MSA
27057,353338.8,331326.0,330153.6,340757.4,366151.4,385451.5,413026.5,445123.5,475416.6,500710.1,SF_MSA


In [42]:
df_GDP= df_gdp

In [43]:
df_GDP = df_GDP.melt(id_vars=['MSA'], 
        var_name="Year", 
        value_name="GDP_mn$")
df_GDP['Year']=df_GDP['Year'].astype(int)
df_GDP['GDP_mn$']=df_GDP['GDP_mn$'].astype(float)
df_GDP.set_index(['MSA', 'Year'], inplace = True, drop = True)
df_GDP.head()

,,GDP_mn$
MSA,Year,
LA_MSA,2008,772747.1
NY_MSA,2008,1257147.7
SF_MSA,2008,353338.8
LA_MSA,2009,741630.0
NY_MSA,2009,1276469.8


In [44]:
df_GDP.shape, df_Airtraffic.shape, dfp.shape, dfi.shape, df_AirQuality.shape

((30, 1), (33, 1), (30, 1), (30, 1), (33, 4))

In [45]:
list_df = [df_GDP, dfi, dfp,  df_Airtraffic,df_AirQuality]
df_final = df_GDP
for frame in list_df[1:]:
    df_final = pd.merge(df_final, frame, left_index = True, right_index = True, how = 'outer')
df_final.head()

GDP_mn$  Per Capita Income  Population  Passengers  % Good Days  \
MSA    Year                                                                     
LA_MSA 2008  772747.1            46729.5  12692740.0  36273289.0    35.974499   
       2009  741630.0            44341.0  12774577.0  34179598.0    39.726027   
       2010  763975.9            46654.5  12841606.0  35475341.0    38.492019   
       2011  782565.1            48954.0  12939463.0  37011731.0    36.255708   
       2012  820862.6            52377.5  13041538.0  37790767.0    42.531876   

             90th Percentile AQI   Good Days  Median AQI  
MSA    Year                                               
LA_MSA 2008           115.666667  131.666667   63.333333  
       2009           103.333333  145.000000   58.333333  
       2010            96.333333  140.000000   57.333333  
       2011           100.000000  132.333333   59.000000  
       2012            97.666667  155.666667   58.000000

In [46]:
df_final.reset_index(inplace = True)

In [47]:
df_final

,MSA,Year,GDP_mn$,Per Capita Income,Population,Passengers,% Good Days,90th Percentile AQI,Good Days,Median AQI
0,LA_MSA,2008,772747.1,46729.5,12692740.0,36273289.0,35.974499,115.666667,131.666667,63.333333
1,LA_MSA,2009,741630.0,44341.0,12774577.0,34179598.0,39.726027,103.333333,145.000000,58.333333
2,LA_MSA,2010,763975.9,46654.5,12841606.0,35475341.0,38.492019,96.333333,140.000000,57.333333
3,LA_MSA,2011,782565.1,48954.0,12939463.0,37011731.0,36.255708,100.000000,132.333333,59.000000
4,LA_MSA,2012,820862.6,52377.5,13041538.0,37790767.0,42.531876,97.666667,155.666667,58.000000
5,LA_MSA,2013,852034.2,51802.0,13132253.0,38908822.0,44.200913,88.666667,161.333333,54.666667
6,LA_MSA,2014,901979.9,54620.0,13209445.0,40931688.0,39.178082,93.666667,143.000000,58.666667
7,LA_MSA,2015,967100.1,58272.0,13283824.0,43464589.0,42.009132,98.000000,153.333333,57.000000
8,LA_MSA,2016,996431.8,59807.0,13328261.0,46920468.0,46.539162,90.666667,170.333333,55.000000
9,LA_MSA,2017,1043735.1,61909.5,13353907.0,48525247.0,47.397260,100.333333,173.000000,55.000000


# Creating SQLite database table using SQLAlchemy

In [48]:
import sqlalchemy
from sqlalchemy import create_engine
# from sqlalchemy.orm import relationship
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy import Column, Integer, String, Float, ForeignKey, DateTime, Boolean, Numeric
# from sqlalchemy.orm import relationship, backref
# from sqlalchemy.orm import sessionmaker

In [49]:
#Creating a sqllite database 'database.db' in current directory
engine = create_engine('sqlite:///database.db', echo=False)

#engine = create_engine('sqlite://', echo=False)  # to use for in memory database

In [51]:
engine.execute('DROP TABLE IF EXISTS plotting_data;')  # Drop existing table before creating new table

# writing pandas dataframe directly to sqllite table 'plotting_data'
df_final.to_sql('plotting_data', con = engine, index = False) 

In [52]:
# run a SQL query
engine.execute('SELECT * FROM plotting_data').fetchall()

[('LA_MSA', 2008, 772747.1, 46729.5, 12692740.0, 36273289.0, 35.97449908925319, 115.66666666666667, 131.66666666666666, 63.333333333333336),
 ('LA_MSA', 2009, 741630.0, 44341.0, 12774577.0, 34179598.0, 39.726027397260275, 103.33333333333333, 145.0, 58.333333333333336),
 ('LA_MSA', 2010, 763975.9, 46654.5, 12841606.0, 35475341.0, 38.492018566738366, 96.33333333333333, 140.0, 57.333333333333336),
 ('LA_MSA', 2011, 782565.1, 48954.0, 12939463.0, 37011731.0, 36.25570776255708, 100.0, 132.33333333333334, 59.0),
 ('LA_MSA', 2012, 820862.6, 52377.5, 13041538.0, 37790767.0, 42.53187613843352, 97.66666666666667, 155.66666666666666, 58.0),
 ('LA_MSA', 2013, 852034.2, 51802.0, 13132253.0, 38908822.0, 44.20091324200913, 88.66666666666667, 161.33333333333334, 54.666666666666664),
 ('LA_MSA', 2014, 901979.9, 54620.0, 13209445.0, 40931688.0, 39.178082191780824, 93.66666666666667, 143.0, 58.666666666666664),
 ('LA_MSA', 2015, 967100.1, 58272.0, 13283824.0, 43464589.0, 42.009132420091326, 98.0, 153.333